In [1]:
# import necessary libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin

import chess.pgn
import re

A link to the whole project including the raw data at dox.abv.bg [link](https://dox.abv.bg/download?id=de3fbe797a#)

### Data Acquisition

The chess games used for this analysis were acquired from three different sources:
* lichess.org - The games were obtained from the lichess open database found at: [link](https://database.lichess.org/). The games in the database are organized in files for each month. They are in the pgn.zst file format which allows a partial(not completely finished) download to be decompressed. This is useful since the original files are quite large - about 30 gigabytes each.  A part of the games from the month of May, 2024 were manually downloaded. After decompression the data is a pgn file (portable game notation)- a standard format for chess games storage.
* chess.com - The games were obtained from a collection of games from the weekly tournament called Titled Tuesday where only strong titled players can participate. The collection was created by Chess Nerd. The games are split in files corresponding to each edition of the tournament. The games were downloaded with a script and appended to one big pgn file. Only games from this year(2024) were included. The collection by Chess Nerd can be found at the following [link](https://chessnerd.net/pgn/chesscom/titled-tuesday/).
* Top Chess Engine Championship(TCEC). The games were manually downloaded from this webpage [link](https://github.com/TCEC-Chess/tcecgames/releases). The file downloaded is listed in the assets  for the final of season 26 as "TCEC-all-in-one-compact.zip". 

The following functions were used to download the games played on Chess.com and convert them to one big pgn file.

In [2]:
def download_specific_files_from_url(url, download_folder, prefix = "24"):
    # Make the request to the URL
    response = requests.get(url)
    # Check that the request was successful
    response.raise_for_status()

    # Parse the HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # Find all links that end with ".pgn" and start with the given prefix (the prefix is the year)
    links = soup.find_all('a')
    pgn_links = [link["href"] for link in links if link["href"].endswith(".pgn") and os.path.basename(link["href"]).startswith(prefix)]

    # Ensure download folder exists
    current_directory = os.getcwd()
    subdirectory_path = os.path.join(current_directory, download_folder)
    if not os.path.exists(subdirectory_path):
        os.makedirs(subdirectory_path)

    # Download each pgn file that matches the prefix
    for pgn_link in pgn_links:
        # Construct the full URL 
        file_url = urljoin(url, pgn_link)
        file_name = os.path.join(subdirectory_path, os.path.basename(pgn_link))
        print(f"Downloading {file_url} to {file_name}")
        response = requests.get(file_url)
        # Check for successful download
        response.raise_for_status()  
        with open(file_name, 'wb') as file:
            file.write(response.content)

    print("Download Complete")

In [3]:
def merge_pgn_files(source_folder, output_file):
    pgn_files = [file for file in os.listdir(source_folder) if file.endswith("pgn")]

    with open(output_file, "w") as outfile:
        for pgn_file in pgn_files:
            file_path = os.path.join(source_folder, pgn_file)
            with open(file_path, "r") as infile:
                outfile.write(infile.read())
                # Add a blank line between games
                outfile.write("\n\n")

In [4]:
# Download chess.com games
url = "https://chessnerd.net/pgn/chesscom/titled-tuesday/"
download_folder = "downloaded_chesscom_pgns"
#download_specific_files_from_url(url, download_folder, prefix = "24")

In [5]:
output_file = "raw_data/merged_chesscom_games.pgn"
# merge_pgn_files(download_folder, output_file)

### Data Conversion

The following function was used the convert the raw pgn files into csv files:

In [6]:
def convert_pgn_to_csv(
    pgn_file_path,
    output_file_path,
    number_of_games = None, 
    chunk_size = 100, 
    print_progress = False
    ):
    with open(pgn_file_path) as source_file:
        game_counter = 0
        games = []
        while True:
            game = chess.pgn.read_game(source_file)
            if game is None:
                break
            game_info = {
                "event": game.headers.get("Event"),
                "site": game.headers.get("Site"),
                "date": game.headers.get("Date"),
                "round": game.headers.get("Round"),
                "white": game.headers.get("White"),
                "black": game.headers.get("Black"),
                "result": game.headers.get("Result"),
                "utc_date": game.headers.get("UTCDate"),
                "utc_time": game.headers.get("UTCTime"),
                "white_elo": game.headers.get("WhiteElo"),
                "black_elo": game.headers.get("BlackElo"),
                "eco": game.headers.get("ECO"),
                "opening": game.headers.get("Opening"),
                "time_control": game.headers.get("TimeControl"),
                "termination": game.headers.get("Termination"),
                "moves": game.board().variation_san(game.mainline_moves())
            }
            games.append(game_info)
            game_counter += 1

            if game_counter % chunk_size == 0:
                if(print_progress):
                    print(f"Processed {game_counter} games")
                df = pd.DataFrame(games)
                if game_counter == chunk_size:
                    df.to_csv(output_file_path, mode = "w", header = True, index = False)
                else:
                    df.to_csv(output_file_path, mode = "a", header = False, index = False)
                games = []
            
            if number_of_games is not None and game_counter >= number_of_games:
                break
        # Write any remainig games
        if games:
            df = pd.DataFrame(games)
            df.to_csv(output_file_path, mode = "a", header = False, index = False)           
    
    print(f"Processed a total of {game_counter} games. CSV file saved to {output_file_path}")    

In [7]:
# Convert lichess games to csv
number_of_games = 205000
pgn_file_path = "raw_data/lichess_05_2024.pgn"
output_file_path = "data/lichess_05_2024.csv"
# convert_pgn_to_csv(pgn_file_path, output_file_path, number_of_games, chunk_size = 100, print_progress = True)

There is a slight issue with reproducibility in relation to the lichess games. To reproduce the same results one needs to make sure at least 205 000 games have been partially downloaded as pgn. For reference the raw data of the partial download is 1.22 gigabytes on my machine.

In [8]:
# Convert chess.com Titled Tuesday 2024 games to csv
pgn_file_path = "raw_data/merged_chesscom_games.pgn"
output_file_path = "data/chesscom_2024.csv"
# convert_pgn_to_csv(pgn_file_path, output_file_path, chunk_size = 100, print_progress = True)

In [9]:
# Convert engine games to csv
pgn_file_path = "raw_data/TCEC-everything.pgn"
output_file_path = "data/engine_games.csv"
# convert_pgn_to_csv(pgn_file_path, output_file_path, chunk_size = 100, print_progress = True)

### Data Cleaning and Exploration

#### Cleaning lichess games data:

In [10]:
lichess_data = pd.read_csv("data/lichess_05_2024.csv")

In [11]:
lichess_data.memory_usage().sum()/(1024 * 1024)

25.024539947509766

In [12]:
lichess_data.shape

(205000, 16)

In [13]:
lichess_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205000 entries, 0 to 204999
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   event         205000 non-null  object
 1   site          205000 non-null  object
 2   date          205000 non-null  object
 3   round         205000 non-null  object
 4   white         205000 non-null  object
 5   black         205000 non-null  object
 6   result        205000 non-null  object
 7   utc_date      205000 non-null  object
 8   Utc_time      205000 non-null  object
 9   whitee_elo    205000 non-null  int64 
 10  black_elo     205000 non-null  int64 
 11  eco           205000 non-null  object
 12  opening       205000 non-null  object
 13  time_control  205000 non-null  object
 14  termination   205000 non-null  object
 15  moves         204522 non-null  object
dtypes: int64(2), object(14)
memory usage: 25.0+ MB


Remove the columns I will not be using:

In [14]:
lichess_data_clean = lichess_data.drop(columns = ["event", "site", "round", "utc_date", "Utc_time"])

Convert date to datetime:

In [15]:
lichess_data_clean["date"] = pd.to_datetime(lichess_data_clean["date"])

In [16]:
lichess_data_clean.dtypes

date            datetime64[ns]
white                   object
black                   object
result                  object
whitee_elo               int64
black_elo                int64
eco                     object
opening                 object
time_control            object
termination             object
moves                   object
dtype: object

In [17]:
lichess_data_clean = lichess_data_clean.drop_duplicates()

In [18]:
lichess_data_clean.shape

(205000, 11)

In [19]:
lichess_data_clean = lichess_data_clean.dropna(subset = ["moves"])

In [20]:
lichess_data_clean.shape

(204522, 11)

Fix my typos:

In [21]:
lichess_data_clean = lichess_data_clean.rename(columns = {'whitee_elo': 'white_elo'})

In [22]:
lichess_data_clean

,date,white,black,result,white_elo,black_elo,eco,opening,time_control,termination,moves
0,2024-05-01,julio_arias2,Nanaldopark,0-1,1309,1364,C40,King's Pawn Game: McConnell Defense,600+0,Normal,1. e4 e5 2. Nf3 Qf6 3. Bc4 Bc5 4. O-O d6 5. Nc...
1,2024-05-01,tbecalli,Adriano_BSB,0-1,1568,1904,B27,Sicilian Defense: Hyperaccelerated Dragon,300+0,Normal,1. e4 c5 2. Nf3 g6 3. Bc4 Bg7 4. Nc3 e6 5. O-O...
2,2024-05-01,Joao_Almeida2005,Kauani_Plnr,1-0,1918,1655,B00,King's Pawn Game,300+0,Time forfeit,1. e4
5,2024-05-01,OLIVEIRA13,jckawagoe,0-1,1903,2147,C54,"Italian Game: Classical Variation, Giuoco Pian...",300+0,Normal,1. e4 e5 2. Nf3 Nc6 3. Bc4 Nf6 4. d3 Bc5 5. c3...
6,2024-05-01,enrique191060,electronicmoon,0-1,1842,1589,B10,Caro-Kann Defense,1800+0,Normal,1. e4 c6 2. c3 d5 3. e5 Bf5 4. d4 e6 5. Nf3 c5...
...,...,...,...,...,...,...,...,...,...,...,...
204995,2024-05-01,iLorenz,ILUVLIVYYY,1-0,1388,1335,B06,Modern Defense,180+0,Time forfeit,1. e4 g6 2. d4 Bg7 3. Nf3 d6 4. Be3 Nc6 5. Bb5...
204996,2024-05-01,hulivar,Andre_uk,1-0,1291,1234,B06,Modern Defense,180+0,Time forfeit,1. e4 g6 2. Nf3 Bg7 3. d4 e6 4. Bd3 Ne7 5. O-O...
204997,2024-05-01,pradyumna_1,aprasher,0-1,1135,1143,C50,"Italian Game: Giuoco Pianissimo, Normal",180+0,Normal,1. e4 e5 2. Bc4 Nc6 3. d3 Nf6 4. Nf3 Bc5 5. Ng...
204998,2024-05-01,CLADY,Mazinger3000,1-0,1860,1884,C30,King's Gambit Declined: Queen's Knight Defense,180+0,Time forfeit,1. e4 Nc6 2. f4 e5 3. fxe5 Nxe5 4. Nf3 Ng6 5. ...


Add a column "rating_difference" I will be using in the analysis:

In [23]:
lichess_data_clean["rating_difference"] = lichess_data_clean["white_elo"] -  lichess_data_clean["black_elo"]

In [24]:
lichess_data_clean

,date,white,black,result,white_elo,black_elo,eco,opening,time_control,termination,moves,rating_difference
0,2024-05-01,julio_arias2,Nanaldopark,0-1,1309,1364,C40,King's Pawn Game: McConnell Defense,600+0,Normal,1. e4 e5 2. Nf3 Qf6 3. Bc4 Bc5 4. O-O d6 5. Nc...,-55
1,2024-05-01,tbecalli,Adriano_BSB,0-1,1568,1904,B27,Sicilian Defense: Hyperaccelerated Dragon,300+0,Normal,1. e4 c5 2. Nf3 g6 3. Bc4 Bg7 4. Nc3 e6 5. O-O...,-336
2,2024-05-01,Joao_Almeida2005,Kauani_Plnr,1-0,1918,1655,B00,King's Pawn Game,300+0,Time forfeit,1. e4,263
5,2024-05-01,OLIVEIRA13,jckawagoe,0-1,1903,2147,C54,"Italian Game: Classical Variation, Giuoco Pian...",300+0,Normal,1. e4 e5 2. Nf3 Nc6 3. Bc4 Nf6 4. d3 Bc5 5. c3...,-244
6,2024-05-01,enrique191060,electronicmoon,0-1,1842,1589,B10,Caro-Kann Defense,1800+0,Normal,1. e4 c6 2. c3 d5 3. e5 Bf5 4. d4 e6 5. Nf3 c5...,253
...,...,...,...,...,...,...,...,...,...,...,...,...
204995,2024-05-01,iLorenz,ILUVLIVYYY,1-0,1388,1335,B06,Modern Defense,180+0,Time forfeit,1. e4 g6 2. d4 Bg7 3. Nf3 d6 4. Be3 Nc6 5. Bb5...,53
204996,2024-05-01,hulivar,Andre_uk,1-0,1291,1234,B06,Modern Defense,180+0,Time forfeit,1. e4 g6 2. Nf3 Bg7 3. d4 e6 4. Bd3 Ne7 5. O-O...,57
204997,2024-05-01,pradyumna_1,aprasher,0-1,1135,1143,C50,"Italian Game: Giuoco Pianissimo, Normal",180+0,Normal,1. e4 e5 2. Bc4 Nc6 3. d3 Nf6 4. Nf3 Bc5 5. Ng...,-8
204998,2024-05-01,CLADY,Mazinger3000,1-0,1860,1884,C30,King's Gambit Declined: Queen's Knight Defense,180+0,Time forfeit,1. e4 Nc6 2. f4 e5 3. fxe5 Nxe5 4. Nf3 Ng6 5. ...,-24


Add a column to store the outcome of the game as an integer: 1 if white wins, zero if the games ends in a draw and -1 if black wins:

In [25]:
lichess_data_clean["result"].value_counts()

result
1-0        102303
0-1         94774
1/2-1/2      7430
*              15
Name: count, dtype: int64

Remove result values containing disconnects:

In [26]:
lichess_data_clean = lichess_data_clean[lichess_data_clean["result"] != "*"] 

In [27]:
lichess_data_clean["result"].value_counts()

result
1-0        102303
0-1         94774
1/2-1/2      7430
Name: count, dtype: int64

Define function to map string game results to a numerical result:

In [28]:
def map_string_result_to_numerical_result(value):
    if value == "1-0":
        return 1
    elif value == "0-1":
        return -1
    elif value == "1/2-1/2":
        return 0
    else: return None

In [29]:
lichess_data_clean.loc[:,"result_num"] = lichess_data_clean["result"].apply(map_string_result_to_numerical_result)
# lichess_data_clean["result_num"] = lichess_data_clean["result"].apply(map_string_result_to_numerical_result)

C:\Users\Sasor1\AppData\Local\Temp\ipykernel_7032\2710212476.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lichess_data_clean.loc[:,"result_num"] = lichess_data_clean["result"].apply(map_string_result_to_numerical_result)


In [30]:
lichess_data_clean["result_num"].value_counts()

result_num
 1    102303
-1     94774
 0      7430
Name: count, dtype: int64

In [31]:
lichess_data_clean["result"] = lichess_data_clean["result"].astype("category")

C:\Users\Sasor1\AppData\Local\Temp\ipykernel_7032\419885391.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lichess_data_clean["result"] = lichess_data_clean["result"].astype("category")


In [32]:
lichess_data_clean.dtypes

date                 datetime64[ns]
white                        object
black                        object
result                     category
white_elo                     int64
black_elo                     int64
eco                          object
opening                      object
time_control                 object
termination                  object
moves                        object
rating_difference             int64
result_num                    int64
dtype: object

In [33]:
lichess_data_clean

,date,white,black,result,white_elo,black_elo,eco,opening,time_control,termination,moves,rating_difference,result_num
0,2024-05-01,julio_arias2,Nanaldopark,0-1,1309,1364,C40,King's Pawn Game: McConnell Defense,600+0,Normal,1. e4 e5 2. Nf3 Qf6 3. Bc4 Bc5 4. O-O d6 5. Nc...,-55,-1
1,2024-05-01,tbecalli,Adriano_BSB,0-1,1568,1904,B27,Sicilian Defense: Hyperaccelerated Dragon,300+0,Normal,1. e4 c5 2. Nf3 g6 3. Bc4 Bg7 4. Nc3 e6 5. O-O...,-336,-1
2,2024-05-01,Joao_Almeida2005,Kauani_Plnr,1-0,1918,1655,B00,King's Pawn Game,300+0,Time forfeit,1. e4,263,1
5,2024-05-01,OLIVEIRA13,jckawagoe,0-1,1903,2147,C54,"Italian Game: Classical Variation, Giuoco Pian...",300+0,Normal,1. e4 e5 2. Nf3 Nc6 3. Bc4 Nf6 4. d3 Bc5 5. c3...,-244,-1
6,2024-05-01,enrique191060,electronicmoon,0-1,1842,1589,B10,Caro-Kann Defense,1800+0,Normal,1. e4 c6 2. c3 d5 3. e5 Bf5 4. d4 e6 5. Nf3 c5...,253,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
204995,2024-05-01,iLorenz,ILUVLIVYYY,1-0,1388,1335,B06,Modern Defense,180+0,Time forfeit,1. e4 g6 2. d4 Bg7 3. Nf3 d6 4. Be3 Nc6 5. Bb5...,53,1
204996,2024-05-01,hulivar,Andre_uk,1-0,1291,1234,B06,Modern Defense,180+0,Time forfeit,1. e4 g6 2. Nf3 Bg7 3. d4 e6 4. Bd3 Ne7 5. O-O...,57,1
204997,2024-05-01,pradyumna_1,aprasher,0-1,1135,1143,C50,"Italian Game: Giuoco Pianissimo, Normal",180+0,Normal,1. e4 e5 2. Bc4 Nc6 3. d3 Nf6 4. Nf3 Bc5 5. Ng...,-8,-1
204998,2024-05-01,CLADY,Mazinger3000,1-0,1860,1884,C30,King's Gambit Declined: Queen's Knight Defense,180+0,Time forfeit,1. e4 Nc6 2. f4 e5 3. fxe5 Nxe5 4. Nf3 Ng6 5. ...,-24,1


Below I will define a function to count the number of moves in each game. Then I will create a number_of_moves_ column.

In [34]:
def count_moves(moves):
    # Split by '.' to identify full moves, then subtract 1 (for index starting at 1)
    return len(moves.split('.')) - 1

In [35]:
lichess_data_clean.loc[:, "number_of_moves"] = lichess_data_clean["moves"].apply(count_moves)
#lichess_data_clean["number_of_moves"] = lichess_data_clean["moves"].apply(count_moves) 

C:\Users\Sasor1\AppData\Local\Temp\ipykernel_7032\93758785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lichess_data_clean.loc[:, "number_of_moves"] = lichess_data_clean["moves"].apply(count_moves)


In [36]:
lichess_data_clean.columns

Index(['date', 'white', 'black', 'result', 'white_elo', 'black_elo', 'eco',
       'opening', 'time_control', 'termination', 'moves', 'rating_difference',
       'result_num', 'number_of_moves'],
      dtype='object')

Rearange columns:

In [37]:
columns= list(lichess_data_clean.columns.values)
columns

['date',
 'white',
 'black',
 'result',
 'white_elo',
 'black_elo',
 'eco',
 'opening',
 'time_control',
 'termination',
 'moves',
 'rating_difference',
 'result_num',
 'number_of_moves']

In [38]:
new_order = [
'date',
'white',
'black',
'result',
'result_num',
'white_elo',
'black_elo',
'rating_difference',
'moves',
'number_of_moves',
'eco',
'opening',
'time_control',
'termination'
]

In [39]:
lichess_data_clean = lichess_data_clean[new_order]
del columns
del new_order
lichess_data_clean

,date,white,black,result,result_num,white_elo,black_elo,rating_difference,moves,number_of_moves,eco,opening,time_control,termination
0,2024-05-01,julio_arias2,Nanaldopark,0-1,-1,1309,1364,-55,1. e4 e5 2. Nf3 Qf6 3. Bc4 Bc5 4. O-O d6 5. Nc...,21,C40,King's Pawn Game: McConnell Defense,600+0,Normal
1,2024-05-01,tbecalli,Adriano_BSB,0-1,-1,1568,1904,-336,1. e4 c5 2. Nf3 g6 3. Bc4 Bg7 4. Nc3 e6 5. O-O...,78,B27,Sicilian Defense: Hyperaccelerated Dragon,300+0,Normal
2,2024-05-01,Joao_Almeida2005,Kauani_Plnr,1-0,1,1918,1655,263,1. e4,1,B00,King's Pawn Game,300+0,Time forfeit
5,2024-05-01,OLIVEIRA13,jckawagoe,0-1,-1,1903,2147,-244,1. e4 e5 2. Nf3 Nc6 3. Bc4 Nf6 4. d3 Bc5 5. c3...,33,C54,"Italian Game: Classical Variation, Giuoco Pian...",300+0,Normal
6,2024-05-01,enrique191060,electronicmoon,0-1,-1,1842,1589,253,1. e4 c6 2. c3 d5 3. e5 Bf5 4. d4 e6 5. Nf3 c5...,55,B10,Caro-Kann Defense,1800+0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204995,2024-05-01,iLorenz,ILUVLIVYYY,1-0,1,1388,1335,53,1. e4 g6 2. d4 Bg7 3. Nf3 d6 4. Be3 Nc6 5. Bb5...,34,B06,Modern Defense,180+0,Time forfeit
204996,2024-05-01,hulivar,Andre_uk,1-0,1,1291,1234,57,1. e4 g6 2. Nf3 Bg7 3. d4 e6 4. Bd3 Ne7 5. O-O...,31,B06,Modern Defense,180+0,Time forfeit
204997,2024-05-01,pradyumna_1,aprasher,0-1,-1,1135,1143,-8,1. e4 e5 2. Bc4 Nc6 3. d3 Nf6 4. Nf3 Bc5 5. Ng...,21,C50,"Italian Game: Giuoco Pianissimo, Normal",180+0,Normal
204998,2024-05-01,CLADY,Mazinger3000,1-0,1,1860,1884,-24,1. e4 Nc6 2. f4 e5 3. fxe5 Nxe5 4. Nf3 Ng6 5. ...,32,C30,King's Gambit Declined: Queen's Knight Defense,180+0,Time forfeit


In [40]:
lichess_data_clean["time_control"].value_counts()

time_control
60+0       59935
180+0      40534
300+0      27691
600+0      21022
180+2      17063
           ...  
900+45         1
45+15          1
1500+25        1
600+120        1
420+4          1
Name: count, Length: 274, dtype: int64

Some of the games could contain engine evalutions together with the moves. This could create problems in future processing because such games will not fit the general format. Below I define a function to check for such cases. Then, I will remove such games from the dateset it they exist.

In [41]:
def contains_engine_evaluations(pgn_text):
    # Pattern to detect engine evaluations in PGN
    evaluation_pattern = r'\[%eval'
    return bool(re.search(evaluation_pattern, pgn_text))

In [42]:
lichess_data_clean["moves"].apply(contains_engine_evaluations).sum()

0

There are no games containing engine evaluations in the dataset.

Resetting indexes:

In [43]:
lichess_data_clean = lichess_data_clean.reset_index(drop = True)

In [44]:
lichess_data_clean

,date,white,black,result,result_num,white_elo,black_elo,rating_difference,moves,number_of_moves,eco,opening,time_control,termination
0,2024-05-01,julio_arias2,Nanaldopark,0-1,-1,1309,1364,-55,1. e4 e5 2. Nf3 Qf6 3. Bc4 Bc5 4. O-O d6 5. Nc...,21,C40,King's Pawn Game: McConnell Defense,600+0,Normal
1,2024-05-01,tbecalli,Adriano_BSB,0-1,-1,1568,1904,-336,1. e4 c5 2. Nf3 g6 3. Bc4 Bg7 4. Nc3 e6 5. O-O...,78,B27,Sicilian Defense: Hyperaccelerated Dragon,300+0,Normal
2,2024-05-01,Joao_Almeida2005,Kauani_Plnr,1-0,1,1918,1655,263,1. e4,1,B00,King's Pawn Game,300+0,Time forfeit
3,2024-05-01,OLIVEIRA13,jckawagoe,0-1,-1,1903,2147,-244,1. e4 e5 2. Nf3 Nc6 3. Bc4 Nf6 4. d3 Bc5 5. c3...,33,C54,"Italian Game: Classical Variation, Giuoco Pian...",300+0,Normal
4,2024-05-01,enrique191060,electronicmoon,0-1,-1,1842,1589,253,1. e4 c6 2. c3 d5 3. e5 Bf5 4. d4 e6 5. Nf3 c5...,55,B10,Caro-Kann Defense,1800+0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204502,2024-05-01,iLorenz,ILUVLIVYYY,1-0,1,1388,1335,53,1. e4 g6 2. d4 Bg7 3. Nf3 d6 4. Be3 Nc6 5. Bb5...,34,B06,Modern Defense,180+0,Time forfeit
204503,2024-05-01,hulivar,Andre_uk,1-0,1,1291,1234,57,1. e4 g6 2. Nf3 Bg7 3. d4 e6 4. Bd3 Ne7 5. O-O...,31,B06,Modern Defense,180+0,Time forfeit
204504,2024-05-01,pradyumna_1,aprasher,0-1,-1,1135,1143,-8,1. e4 e5 2. Bc4 Nc6 3. d3 Nf6 4. Nf3 Bc5 5. Ng...,21,C50,"Italian Game: Giuoco Pianissimo, Normal",180+0,Normal
204505,2024-05-01,CLADY,Mazinger3000,1-0,1,1860,1884,-24,1. e4 Nc6 2. f4 e5 3. fxe5 Nxe5 4. Nf3 Ng6 5. ...,32,C30,King's Gambit Declined: Queen's Knight Defense,180+0,Time forfeit


In [45]:
lichess_data_clean.dtypes

date                 datetime64[ns]
white                        object
black                        object
result                     category
result_num                    int64
white_elo                     int64
black_elo                     int64
rating_difference             int64
moves                        object
number_of_moves               int64
eco                          object
opening                      object
time_control                 object
termination                  object
dtype: object

In [46]:
lichess_data_clean.to_csv("clean_data/lichess_data.csv")

In [47]:
del lichess_data
del lichess_data_clean

#### Cleaning chess.com games data:

The overall process will be the same, so I will only describe the diffences with the handling of lichess data.

In [48]:
chesscom_data = pd.read_csv("data/chesscom_2024.csv")

In [49]:
chesscom_data.memory_usage().sum()/(1024 * 1024)

16.575931549072266

In [50]:
chesscom_data.shape

(135789, 16)

In [51]:
chesscom_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135789 entries, 0 to 135788
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   event         135789 non-null  object 
 1   site          135789 non-null  object 
 2   date          135789 non-null  object 
 3   round         135789 non-null  int64  
 4   white         135789 non-null  object 
 5   black         135789 non-null  object 
 6   result        135789 non-null  object 
 7   utc_date      0 non-null       float64
 8   utc_time      0 non-null       float64
 9   white_elo     135789 non-null  int64  
 10  black_elo     135789 non-null  int64  
 11  eco           0 non-null       float64
 12  opening       0 non-null       float64
 13  time_control  131398 non-null  object 
 14  termination   121722 non-null  object 
 15  moves         135098 non-null  object 
dtypes: float64(4), int64(3), object(9)
memory usage: 16.6+ MB


In [52]:
chesscom_data.columns

Index(['event', 'site', 'date', 'round', 'white', 'black', 'result',
       'utc_date', 'utc_time', 'white_elo', 'black_elo', 'eco', 'opening',
       'time_control', 'termination', 'moves'],
      dtype='object')

Here I will also drop the columns:"eco" and "opening" since the source files contained no information for them and all the values in them are null. The column "time_control" will also be dropped because it contains the same (wrong) value in all rows.

In [53]:
chesscom_data_clean = chesscom_data.drop(columns = ["event", "site", "round", "utc_date", "utc_time", "eco", "opening", "time_control"])

In [54]:
chesscom_data_clean["date"] = pd.to_datetime(chesscom_data_clean["date"]) 

In [55]:
chesscom_data_clean.dtypes

date           datetime64[ns]
white                  object
black                  object
result                 object
white_elo               int64
black_elo               int64
termination            object
moves                  object
dtype: object

In [56]:
chesscom_data_clean = chesscom_data_clean.drop_duplicates()

In [57]:
chesscom_data_clean.shape

(135789, 8)

In [58]:
chesscom_data_clean["moves"].isna().sum()

691

In [59]:
chesscom_data_clean = chesscom_data_clean.dropna(subset = ["moves"])

In [60]:
chesscom_data_clean.columns

Index(['date', 'white', 'black', 'result', 'white_elo', 'black_elo',
       'termination', 'moves'],
      dtype='object')

In [61]:
chesscom_data_clean["rating_difference"] = chesscom_data_clean["white_elo"] - chesscom_data_clean["black_elo"]

In [62]:
chesscom_data_clean["result"].value_counts()

result
1-0        65381
0-1        57719
1/2-1/2    11998
Name: count, dtype: int64

In [63]:
chesscom_data_clean["result_num"] = chesscom_data_clean["result"].apply(map_string_result_to_numerical_result)

In [64]:
chesscom_data_clean["result_num"].value_counts()

result_num
 1    65381
-1    57719
 0    11998
Name: count, dtype: int64

In [65]:
chesscom_data_clean["result"] = chesscom_data_clean["result"].astype("category")

In [66]:
chesscom_data_clean["number_of_moves"] = chesscom_data_clean["moves"].apply(count_moves)

In [67]:
columns = list(chesscom_data_clean.columns.values)
columns

['date',
 'white',
 'black',
 'result',
 'white_elo',
 'black_elo',
 'termination',
 'moves',
 'rating_difference',
 'result_num',
 'number_of_moves']

In [68]:
new_order = [
 'date',
 'white',
 'black',
 'result',
 'result_num',
 'white_elo',
 'black_elo',
 'rating_difference',
 'moves',
 'number_of_moves',
 'termination',
 ]


In [69]:
chesscom_data_clean = chesscom_data_clean[new_order]
del columns
del new_order

In [70]:
chesscom_data_clean["moves"].apply(contains_engine_evaluations).sum()

0

In [71]:
chesscom_data_clean = chesscom_data_clean.reset_index(drop = True)

In [72]:
chesscom_data_clean.head(5)

,date,white,black,result,result_num,white_elo,black_elo,rating_difference,moves,number_of_moves,termination
0,2024-01-02,"Carlsen, Magnus","Santos Flores, Alberto",1-0,1,3366,2532,834,1. e4 e6 2. b3 d5 3. Bb2 dxe4 4. Nc3 Nf6 5. g4...,22,NaN
1,2024-01-02,"Rangel, Daniel","Nakamura, Hikaru",0-1,-1,2531,3238,-707,1. c4 g6 2. g3 Bg7 3. Bg2 d6 4. Nc3 Nc6 5. e4 ...,37,NaN
2,2024-01-02,Nihal Sarin,"Kambrath, Yannick",1-0,1,3203,2531,672,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...,22,NaN
3,2024-01-02,"Novikov, Evgenij","Vachier-Lagrave, Maxime",0-1,-1,2530,3187,-657,1. d4 Nf6 2. c4 e6 3. Nc3 d5 4. cxd5 exd5 5. B...,36,NaN
4,2024-01-02,"Firouzja, Alireza","Rodriguez Fontecha, Marcos",1-0,1,3160,2530,630,1. d4 d5 2. Bg5 Bf5 3. e3 c6 4. Nd2 Nd7 5. Ngf...,44,NaN


In [73]:
chesscom_data_clean.to_csv("clean_data/chesscom_data.csv")
del chesscom_data
del chesscom_data_clean

#### Cleaning engine games data:

In [74]:
engine_data = pd.read_csv("data/engine_games.csv")

In [75]:
engine_data.shape

(49637, 16)

In [76]:
engine_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49637 entries, 0 to 49636
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   event         49637 non-null  object 
 1   site          49637 non-null  object 
 2   date          49637 non-null  object 
 3   round         49637 non-null  float64
 4   white         49637 non-null  object 
 5   black         49637 non-null  object 
 6   result        49637 non-null  object 
 7   utc_date      0 non-null      float64
 8   utc_time      0 non-null      float64
 9   white_elo     48280 non-null  float64
 10  black_elo     48272 non-null  float64
 11  eco           44424 non-null  object 
 12  opening       49565 non-null  object 
 13  time_control  43577 non-null  object 
 14  termination   37110 non-null  object 
 15  moves         49623 non-null  object 
dtypes: float64(5), object(11)
memory usage: 6.1+ MB


In [77]:
engine_data.columns

Index(['event', 'site', 'date', 'round', 'white', 'black', 'result',
       'utc_date', 'utc_time', 'white_elo', 'black_elo', 'eco', 'opening',
       'time_control', 'termination', 'moves'],
      dtype='object')

In [78]:
engine_data["eco"].isna().sum()

5213

In [79]:
engine_data["opening"].isna().sum()

72

In [80]:
engine_data["time_control"].isna().sum()

6060

In [81]:
engine_data_clean = engine_data.drop(columns = ["event", "site", "round", "utc_date", "utc_time"]) 

In [82]:
engine_data_clean["date"] = pd.to_datetime(engine_data["date"])

In [83]:
engine_data_clean.dtypes

date            datetime64[ns]
white                   object
black                   object
result                  object
white_elo              float64
black_elo              float64
eco                     object
opening                 object
time_control            object
termination             object
moves                   object
dtype: object

In [84]:
engine_data_clean.shape

(49637, 11)

In [85]:
engine_data_clean = engine_data_clean.drop_duplicates()

In [86]:
engine_data_clean.shape

(49555, 11)

In [87]:
engine_data_clean["moves"].isna().sum()

8

In [88]:
engine_data_clean = engine_data_clean.dropna(subset = ["moves"])

In [89]:
engine_data_clean["white_elo"].isna().sum()

1357

In [90]:
engine_data_clean["black_elo"].isna().sum()

1365

In this dataset, contrary to the previous ones there are some missing values for "white_elo" and "black_elo". I will drop the corresponding records.

In [91]:
engine_data_clean = engine_data_clean.dropna(subset = ["white_elo"])

In [92]:
engine_data_clean = engine_data_clean.dropna(subset = ["black_elo"])

In [93]:
engine_data_clean.shape

(48171, 11)

In [94]:
engine_data_clean["rating_difference"] = engine_data_clean["white_elo"] -  engine_data_clean["black_elo"]

In [95]:
engine_data_clean.head(5)

,date,white,black,result,white_elo,black_elo,eco,opening,time_control,termination,moves,rating_difference
796,2011-04-29,Greko 8.0,Zappa Mexico II,1/2-1/2,2501.0,2899.0,A09,Reti opening,5400+30,GUI adjudication,1. Nf3 d5 2. c4 c6 3. e3 Nf6 4. Nc3 a6 5. Be2 ...,-398.0
797,2011-04-29,Junior 12.5,Cuckoo 1.1,1-0,2946.0,2540.0,E14,Queen's Indian,5400+30,GUI adjudication,1. d4 Nf6 2. c4 e6 3. Nf3 b6 4. e3 Bb7 5. Bd3 ...,406.0
798,2011-04-30,Gaviota 0.83,Spark 1.0,0-1,2544.0,2953.0,C80,Ruy Lopez,5400+30,GUI adjudication,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...,-409.0
799,2011-04-30,Hiarcs 13.2,Danasah 4.6,1-0,2959.0,2556.0,C42,Petrov's defence,5400+30,GUI adjudication,1. e4 e5 2. Nf3 Nf6 3. Nxe5 d6 4. Nf3 Nxe4 5. ...,403.0
800,2011-04-30,Francesca 0.18,Protector 1.4,0-1,2572.0,2959.0,D27,QGA,5400+30,GUI adjudication,1. d4 d5 2. c4 c6 3. Nf3 Nf6 4. e3 e6 5. Bd3 d...,-387.0


In [96]:
engine_data_clean["result"].value_counts()

result
1/2-1/2    26167
1-0        15894
0-1         6101
*              9
Name: count, dtype: int64

Remove unexpected game outcomes:

In [97]:
engine_data_clean = engine_data_clean[engine_data_clean["result"] != "*"] 

In [98]:
engine_data_clean["result"].value_counts()

result
1/2-1/2    26167
1-0        15894
0-1         6101
Name: count, dtype: int64

In [99]:
engine_data_clean["result_num"] = engine_data_clean["result"].apply(map_string_result_to_numerical_result)

In [100]:
engine_data_clean["result_num"].value_counts()

result_num
 0    26167
 1    15894
-1     6101
Name: count, dtype: int64

In [101]:
engine_data_clean["result"] = engine_data_clean["result"].astype("category")

In [102]:
engine_data_clean.dtypes

date                 datetime64[ns]
white                        object
black                        object
result                     category
white_elo                   float64
black_elo                   float64
eco                          object
opening                      object
time_control                 object
termination                  object
moves                        object
rating_difference           float64
result_num                    int64
dtype: object

In [103]:
engine_data_clean["white_elo"].value_counts()

white_elo
3601.0    841
3532.0    499
3588.0    397
3589.0    362
3662.0    342
         ... 
3007.0      1
3664.0      1
2932.0      1
3006.0      1
2501.0      1
Name: count, Length: 1209, dtype: int64

Function to check if value is true float or can be safely converted to int:

In [104]:
def has_nonzero_decimal(value):
    return isinstance(value, float) and (value != int(value))

In [105]:
engine_data_clean["white_elo"].apply(has_nonzero_decimal).sum()

0

In [106]:
engine_data_clean["black_elo"].apply(has_nonzero_decimal).sum()

0

In [107]:
engine_data_clean["rating_difference"].apply(has_nonzero_decimal).sum()

0

There are no nonzero values after the decimal point so the columns can be safely converted to int. This is expected because generally listed elo rating is rounded to whole numbers.

In [108]:
engine_data_clean["white_elo"] = engine_data_clean["white_elo"].astype(int)

In [109]:
engine_data_clean["black_elo"] = engine_data_clean["black_elo"].astype(int)

In [110]:
engine_data_clean["rating_difference"] = engine_data_clean["rating_difference"].astype(int)

In [111]:
engine_data_clean["result_num"] = engine_data_clean["result_num"].astype(int)

In [112]:
engine_data_clean.dtypes

date                 datetime64[ns]
white                        object
black                        object
result                     category
white_elo                     int32
black_elo                     int32
eco                          object
opening                      object
time_control                 object
termination                  object
moves                        object
rating_difference             int32
result_num                    int32
dtype: object

In [113]:
engine_data_clean["number_of_moves"] = engine_data_clean["moves"].apply(count_moves)

In [114]:
columns = list(engine_data_clean.columns.values)
columns

['date',
 'white',
 'black',
 'result',
 'white_elo',
 'black_elo',
 'eco',
 'opening',
 'time_control',
 'termination',
 'moves',
 'rating_difference',
 'result_num',
 'number_of_moves']

In [115]:
new_order = [
 'date',
 'white',
 'black',
 'result',
 'result_num',
 'white_elo',
 'black_elo',
 'rating_difference',
 'moves',
 'number_of_moves',
 'eco',
 'opening',
 'time_control',
 'termination',
]

In [116]:
engine_data_clean = engine_data_clean[new_order]
del columns
del new_order

In [117]:
engine_data_clean.head(5)

,date,white,black,result,result_num,white_elo,black_elo,rating_difference,moves,number_of_moves,eco,opening,time_control,termination
796,2011-04-29,Greko 8.0,Zappa Mexico II,1/2-1/2,0,2501,2899,-398,1. Nf3 d5 2. c4 c6 3. e3 Nf6 4. Nc3 a6 5. Be2 ...,100,A09,Reti opening,5400+30,GUI adjudication
797,2011-04-29,Junior 12.5,Cuckoo 1.1,1-0,1,2946,2540,406,1. d4 Nf6 2. c4 e6 3. Nf3 b6 4. e3 Bb7 5. Bd3 ...,43,E14,Queen's Indian,5400+30,GUI adjudication
798,2011-04-30,Gaviota 0.83,Spark 1.0,0-1,-1,2544,2953,-409,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...,61,C80,Ruy Lopez,5400+30,GUI adjudication
799,2011-04-30,Hiarcs 13.2,Danasah 4.6,1-0,1,2959,2556,403,1. e4 e5 2. Nf3 Nf6 3. Nxe5 d6 4. Nf3 Nxe4 5. ...,31,C42,Petrov's defence,5400+30,GUI adjudication
800,2011-04-30,Francesca 0.18,Protector 1.4,0-1,-1,2572,2959,-387,1. d4 d5 2. c4 c6 3. Nf3 Nf6 4. e3 e6 5. Bd3 d...,67,D27,QGA,5400+30,GUI adjudication


In [118]:
engine_data_clean["moves"].apply(contains_engine_evaluations).sum()

0

In [119]:
engine_data_clean = engine_data_clean.reset_index(drop = True)

In [120]:
engine_data_clean.to_csv("clean_data/engine_data.csv")
del engine_data_clean
del engine_data